**Initial setup**

In [ ]:
from google.colab import drive
import pandas as pd
import re
import copy
import pickle
import numpy as np
!pip install hazm
!pip install https://github.com/sobhe/hazm/archive/master.zip --upgrade
drive.mount('/content/drive')
data = pd.read_json('/content/drive/MyDrive/Information Retreival/data.json')



from __future__ import unicode_literals
from hazm import *
import nltk
from nltk.stem import PorterStemmer

stemmer = Stemmer()
normalizer = Normalizer()
lemmatizer = Lemmatizer()
tagger = POSTagger(model='/content/drive/MyDrive/Information Retreival/postagger.model')
chunker = Chunker(model='/content/drive/MyDrive/Information Retreival/chunker.model')
parser = DependencyParser(tagger=tagger, lemmatizer=lemmatizer)





Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/sobhe/hazm/archive/master.zip
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Prepare stopwords**

In [ ]:
Array=[]
stopwords=[]

def prepareStopWords():
  #https://github.com/ziaa/Persian-stopwords-collection/blob/master/Stopwords/Kharazi/Pesian_Stop_Words_List.txt
  global stopwords
 
  
  with open('/content/drive/MyDrive/Information Retreival/stopwords.txt') as file:
    stopwords = [line.rstrip() for line in file]
 


  for i in range(len(stopwords)):
    stopwords[i]=stopwords[i].replace("\u200c", " ").split()



  for i in range(len(stopwords)):
    if i==len(stopwords):
      break


    if len(stopwords[i])==1:
      del stopwords[i]
 
  aux=[]
    # print(stopwords[i]) #list
  for i in range(len(stopwords)):
    for sub in stopwords[i]:
      aux.append(normalizer.normalize(sub))

  stopwords=copy.deepcopy(aux)

  

prepareStopWords()

  

**Preprocess documents**

In [ ]:
def trimDocumentEnd():
  trim="انتهای پیام"

  global Array,stopwords
  Array=[]
  # print("data len",data.shape[1])
  # print("data len",data.shape[1])

  for i in range(data.shape[1]):

    idx=data[i]['content'].find(trim, 0, len(data[i]['content']))
    temp=data[i]['content']
    # print(type(word_tokenize(normalizer.normalize(temp))),len(word_tokenize(normalizer.normalize(temp))))

    # prevlen=len(Array)

    if idx==-1:
      Array.append(temp.replace("\u200c", " ").split())
    else:
      #  Array.append(temp[:idx])
       Array.append(temp[:idx].replace("\u200c", " ").split())
    


  auxArray=[]
  # print("Array len",len(Array))


  for i in range(len(Array)):
  # for i in range(1000):
    if i%(int(len(Array)/100))==0:
      print(i/int(len(Array)/100),"%")
    
    dummy=[]  
    for j in range(len(Array[i])):
      
      a = ':؟?،؛./\#*»«[]()_-!"《》'

      temp=copy.deepcopy(Array[i][j])

      for char in a:
        temp = temp.replace(char, "")

      
      temp=normalizer.normalize(temp)
      
    

      temp=stemmer.stem(temp)

    
      # if i==8:
      #   print("original : ",Array[i][j]," Stemmed : ",temp)
      isStopWord=False

    
      if not(temp in stopwords) and len(temp)>0:
        # if i==8:
        #   print("fate was to append\n")

        
        dummy.append(temp)

      # else:
      #   if i==8:
      #     print("fate was to remove\n")

    
    
    for g in range(len(dummy)-1):
      if dummy[g]=='انتهای' and dummy[g+1]=='پیام':
        del dummy[g]
        del dummy[g]

    auxArray.append(dummy)

   
  
  Array=copy.deepcopy(auxArray)


# trimDocumentEnd()

# np.save('/content/drive/MyDrive/Information Retreival/Array.npy',Array)
Array=np.load('/content/drive/MyDrive/Information Retreival/Array.npy',allow_pickle=True)



**Create posting list**

In [ ]:
postingList={}
vectorList=[]

def createPostingsList():
  global postingList,Array,vectorList

  # print(len(Array))
  # time.sleep(10)

  counter=0
  for docId in range(len(Array)):


    
    
    if docId >= counter * len(Array)/100:
      # print(docId,len(Array))
      print("Indexed ", counter," %")
      counter+=1

  # for docId in range(10):


    for wordIndex in range(len(Array[docId])):

      

      word = Array[docId][wordIndex]
      # print(word,type(word))





      #word is not visited in postinglist yet
      if not (word in postingList):
        postingList[word]=[1,{str(docId):[1,[wordIndex]]}]
      
      #word already visited in postingslist
      else:
        #it is the first time this word is observed in the document i
        if not (str(docId) in postingList[word][1]):
          postingList[word][0]+=1
          postingList[word][1][str(docId)]=[1,[wordIndex]]
        
        #if the word is already observed in the document
        else:
      
          #This line was commented since in the second phase for constructing inverted index, we only care about number of docs containng a term, we ignore repetitions in a dicument
          # postingList[word][0]+=1
          postingList[word][1][str(docId)][0]+=1
          postingList[word][1][str(docId)][1].append(wordIndex)

  N=data.shape[1]

  for term in postingList:
    postingList[term][0]=np.log(N/postingList[term][0])
    
    for docId in postingList[term][1]:
      postingList[term][1][docId][0]=1+np.log(postingList[term][1][docId][0])
  

  counter=0
  #Handle creating vector for each doc
  for docId in range(len(Array)):

        
      if docId >= counter * len(Array)/100:
        # print(docId,len(Array))
        print("Generated vector ", counter," %")
        counter+=1

      vectorList.append({})
      for wordIndex in range(len(Array[docId])):

        word = Array[docId][wordIndex]


        if word in vectorList[docId]:
          vectorList[docId][word]+=1
      
        else:
          vectorList[docId][word]=1
      
      
      temp=0

      for aWordInDoc in vectorList[docId]:
        vectorList[docId][aWordInDoc]*=postingList[aWordInDoc][0]
        # print("+vectorList[docId][aWordInDoc]",vectorList[docId],type(vectorList[docId]))
        temp=temp+vectorList[docId][aWordInDoc]**2
      
      temp=temp**0.5
      
      for aWordInDoc in vectorList[docId]:
        vectorList[docId][aWordInDoc]/=temp



class ChampionEntity:
  def __init__(self,docID,score):
    self.docID = docID
    self.score=score
  
  def __lt__(self,other):
    return self.score <= other.score


def upgradePostingsListToChampionList():
  removeRatio=0.66
  counter=0
  temp=0


    

  for term in postingList:
    if temp >= counter * len(postingList)/100:
       print("Upgraded ", counter," %")
       counter+=1
      
    temp+=1

    theList=[]
    for docID in postingList[term][1]:
      theList.append(ChampionEntity(docID,postingList[term][1][docID][0]))
    
    theList.sort()
    
    for indexToBeRemoved in range(int(np.floor(removeRatio*len(theList)))):
      del postingList[term][1][theList[indexToBeRemoved].docID]



######################################
createPostingsList()
upgradePostingsListToChampionList()

# vectorList=None
# np.save('/content/drive/MyDrive/Information Retreival/phase2PL.npy', postingList) 
# np.save('/content/drive/MyDrive/Information Retreival/phase2VL.npy', vectorList) 

# postingList = np.load('/content/drive/MyDrive/Information Retreival/phase2PL.npy',allow_pickle='TRUE').item()
# vectorList = np.load('/content/drive/MyDrive/Information Retreival/phase2VL.npy',allow_pickle='TRUE').item()





Indexed  0  %
Indexed  1  %
Indexed  2  %
Indexed  3  %
Indexed  4  %
Indexed  5  %
Indexed  6  %
Indexed  7  %
Indexed  8  %
Indexed  9  %
Indexed  10  %
Indexed  11  %
Indexed  12  %
Indexed  13  %
Indexed  14  %
Indexed  15  %
Indexed  16  %
Indexed  17  %
Indexed  18  %
Indexed  19  %
Indexed  20  %
Indexed  21  %
Indexed  22  %
Indexed  23  %
Indexed  24  %
Indexed  25  %
Indexed  26  %
Indexed  27  %
Indexed  28  %
Indexed  29  %
Indexed  30  %
Indexed  31  %
Indexed  32  %
Indexed  33  %
Indexed  34  %
Indexed  35  %
Indexed  36  %
Indexed  37  %
Indexed  38  %
Indexed  39  %
Indexed  40  %
Indexed  41  %
Indexed  42  %
Indexed  43  %
Indexed  44  %
Indexed  45  %
Indexed  46  %
Indexed  47  %
Indexed  48  %
Indexed  49  %
Indexed  50  %
Indexed  51  %
Indexed  52  %
Indexed  53  %
Indexed  54  %
Indexed  55  %
Indexed  56  %
Indexed  57  %
Indexed  58  %
Indexed  59  %
Indexed  60  %
Indexed  61  %
Indexed  62  %
Indexed  63  %
Indexed  64  %
Indexed  65  %
Indexed  66  %
Index

In [ ]:
# print(type(vectorList))
# print(vectorList[0])
# print(type(vectorList[0]))
print(np.ceil(2.5))

3.0


In [ ]:
temp=0

for w in vectorList[0]:
  temp=temp+vectorList[0][w]**2

print(temp)

print(vectorList[0])

1.0000000000000002
{'گزار': 0.004049091564166789, 'خبرگزار': 0.00042039091763819696, 'فارس': 0.00025841152173793384, 'کنفدراسیون': 0.1073637833859817, 'فوتبال': 0.06872130043088373, 'آسیا': 0.2418389280075236, 'AFC': 0.10893063256674096, 'در': 0.0012083668163009422, 'نامه': 0.057621776585005356, 'ا': 0.018250664975146227, 'رسم': 0.07024624852698473, 'فدراسیون': 0.05507725256202035, 'ایر': 0.06299833585525424, 'باشگاه': 0.2086348498501133, 'گیت': 0.43974781625054643, 'پسند': 0.38924168944869236, 'ز': 0.04671374777061462, 'قرعه': 0.24116134717036805, 'کش': 0.21372305883726983, 'فوتسال': 0.4351857853724132, 'را': 0.008528262856925449, 'رسما': 0.116337336338659, 'اعلا': 0.03939328295059411, 'کرد': 0.009245828044365254, 'اساس': 0.04560638489359913, '۲۵': 0.08082901508921284, 'فروردین': 0.10893063256674096, 'ماه': 0.043242140577715076, '۱۴۰۱': 0.07382925669988398, 'مراس': 0.07856565100538283, 'مالز': 0.14832175450958582, 'برگزار': 0.03377074784814854, 'م': 0.008465809653945023, 'بعنو': 0.118

In [ ]:
notQueries=[]
Queries=[]

def parseQuery(query):
  global Queries,notQueries
  i=0
  stillinQuery=False
  negate=False

  while(i<len(query)):
    if query[i]==" ":
      i+=1
      continue


    word=[]
    
    if query[i] != '"' and query[i] != "!":
      # print("normal")
      while query[i] != " ":
        # print("i=",i,query[i])
        word.append(query[i])
        i=i+1
        if i>= len(query):
          break

      i=i+1
      Queries.append(''.join(word))
      continue
    
    if query[i] == '"':

      

      raise ValueError('TAs toldwe will not have phrase query')

      i=i+1
      # print("quotation")


      while query[i] != '"':
        # print("i=",i,query[i])
        word.append(query[i])
        i=i+1

      i=i+1
      Queries.append(''.join(word))
      continue
    
    #After ! there is a space
    if query[i] == '!':
      # print("i=",i,query[i])
      i=i+1
      # print("not case")
      if query[i]==" ":
        # print("i=",i,query[i])
        i=i+1
      
      if query[i] != '"':
        while query[i] != " ":
          word.append(query[i])
          # print("i=",i,query[i])
          i=i+1

          if i>= len(query):
            break
        i=i+1

      else:
        i=i+1
        while query[i] != '"':
          word.append(query[i])
          # print("i=",i,query[i])
          i=i+1

          if i>= len(query):
            break

      i=i+1
      notQueries.append(''.join(word))
      continue



  aux=[]

  for q in range(len(notQueries)):
    word=normalizer.normalize(notQueries[q].strip())

    if not(word in stopwords) and len(word)>0:
       
        a = ':؟?،؛./\#*»«[]()_-!'
     
        for char in a:
          word = word.replace(char, "")
        
        print("1",word)
        word=normalizer.normalize(word)
        print("2",word)
        word=stemmer.stem(word)
        print("3",word)
        aux.append(word)
  

  notQueries=copy.deepcopy(aux)

  aux=[]

  for q in range(len(Queries)):
    word=normalizer.normalize(Queries[q].strip())

    if not(word in stopwords) and len(word)>0:
       
        a = ':؟?،؛./\#*»«[]()_-!'
     
        for char in a:
          word = word.replace(char, "")
        
        word=normalizer.normalize(word)
        word=stemmer.stem(word)
        aux.append(word)
  

  Queries=copy.deepcopy(aux)

  for q in range(len(Queries)):
    # print("prev Queries[q]",Queries[q])
    Queries[q]=Queries[q].replace("\u200c", " ").split()

  for q in range(len(notQueries)):
    # print("prev notQueries[q]",notQueries[q])
    notQueries[q]=notQueries[q].replace("\u200c", " ").split()  
  # for q in range(len(Queries)):
  #   Queries[q]=stemmer.stem(normalizer.normalize(Queries[q].strip()))  

  
  #    if not(temp in stopwords) and len(temp)>0:
  #       # if i==8:
  #       #   print("fate was to append\n")

        
  #       a = ':؟?،؛./\ # *»«[]()_-!'
     

  #       for char in a:
  #         temp = temp.replace(char, "")
        
  #       temp=normalizer.normalize(temp)
  #       temp=stemmer.stem(temp)
  
      #Remove duplicate in query
  aux_query=[]

  for q in Queries:
    duplicate=False

    for damn in aux_query:
      if q==damn:
        duplicate=True
        break
          
    if duplicate==False:
      aux_query.append(q)
      
  Queries=[]
  # print("aux_query",aux_query)

  for i in range(len(aux_query)):
    Queries.append(copy.deepcopy(aux_query[i][0]))
    
  Queries=[]

  for i in range(len(aux_query)):
    Queries.append(aux_query[i][0])
  
 

Query=input("Query")


# for i in range(len(Query)):
#   print(i," ",Query[i])


parseQuery(Query)

# print("modified not Queries : ",notQueries)
# print("modifed Queries : ",Queries)
print()

if len(notQueries)>0:
  raise ValueError('TAs told theres no not query in this phase')

found=0


#Find score for a word in a doc
def giveScore(word,docID,mode):
  docID=str(docID)
  global found

  # print("in giveScore, word details:",type(word),len(word),word)



  if mode=="pos":
    # if len(word)==1:
    if not(word in vectorList[int(docID)]):
      return 0
    else: 
        # return postingList[word][1][docID][0]
      return vectorList[int(docID)][word]

  
   

    # else:
    #   raise Exception('Phrase queries are not allowed between " " ')
    #   found = 0
    #   return backTrack(word,0,docID,0)
      

  # elif mode=="neg":
  #   if len(word)==1:
  #     if not(word[0] in postingList):
  #       return 1
        
  #     if docID in postingList[word[0]][1]:
  #       # return postingList[word][1][docID][0]
  #       return -postingList[word[0]][1][docID][0]
      
  #     else:
  #       return 1

  #   else:
  #     found = 0
  #     temp=backTrack(word,0,docID,0)

  #     if temp>0:
  #       return -temp*2
      
  #     else:
  #       return 1
      

  # else:
  #   raise Exception("Not a valid mode!")


#Handle phrase queries
def backTrack(wordArray,depth,docID,expectedIDX):
  
  docID=str(docID)
  global found



  if depth==0:

    if not(wordArray[0] in postingList):
      return 0

    if docID in postingList[wordArray[0]][1]:

      for occuredFirstPosition in postingList[wordArray[0]][1][docID][1]:
        backTrack(wordArray,depth+1,docID,occuredFirstPosition+1)
    
    return found
      
  elif depth<len(wordArray):
    if not (wordArray[depth] in postingList):
      return 

    if docID in postingList[wordArray[depth]][1] and (expectedIDX in postingList[wordArray[depth]][1][docID][1]):
      backTrack(wordArray,depth+1,docID,expectedIDX+1)
    
  
  elif depth==len(wordArray):
    found=found+1


# LEFT: HANDLE SCORINNG, CHECK DUPLICATE WORD IN QUERY

#Returns a singe scalar, the proximity of doc and query. Query may contain words not in posting list, QueryVector is only for those that are
def scoreWrapper(docID,QueryVector):
  docID=str(docID)

  score=np.zeros_like(QueryVector)
  counter=0

  for i in range(len(Queries)):
    if Queries[i] in postingList:
      score[counter]= giveScore(Queries[i],docID,'pos')
      counter+=1

  # score=score/np.linalg.norm(score)

  # if int(docID) %5==0:
  #   print("Score",score,"QueryVector",QueryVector)
 
  return np.dot(score,QueryVector)

  
  # for i in range(len(notQueries)):
  #   score[i+len(Queries)]= giveScore(notQueries[i],docID,'neg')
  
  # print(score)

  # fullSatisfaction=1

  # for i in range(score.shape[0]):
  #   if score[i]<1:
  #     fullSatisfaction=0
  #     break
  
  # docScore=0
  # null=True

  # for i in range(len(Queries)):
  #   if score[i]>0:
  #     null=False
  #     break

  # if null:
  #   fullSatisfaction=-1

  # if fullSatisfaction==1:
  #   docScore=np.amin(score)*score.shape[0]
  #   score=score-np.amin(score)
  #   docScore+=np.mean(score)

  # elif fullSatisfaction==0:
  #   docScore=np.mean(score)

  # else:
  #   docScore=-100
  
  # return(fullSatisfaction,docScore)


class QueryResponse:
  def __init__(self,docID,score):
    self.docID = docID
    self.score=score
  
  def __lt__(self,other):
    return self.score <= other.score



#retrieve the answers. Assume tf=1 for all query words
def retrieve():

    result=[]

    realQueryLength=0

    for i in range(len(Queries)):
      if Queries[i] in postingList:
         realQueryLength+=1

    QueryVector=np.zeros((realQueryLength))
    counter=0

    for i in range(len(Queries)):
      # print("Queries[i] in rertrieve",Queries[i])
      # print("postingList[Queries[i]][0]",postingList[Queries[i]][0])
      if Queries[i] in postingList:
        QueryVector[counter]=postingList[Queries[i]][0]
        counter+=1
 
    # print("QueryVector Pre normalize ",QueryVector)
    QueryVector/=np.linalg.norm(QueryVector)
    # print("QueryVector Post normalize ",QueryVector)


    docsToBeAnalyzed={}

    for queryWord in Queries:
      if queryWord in postingList:
        for docContainingAWord in postingList[queryWord][1]:
          docsToBeAnalyzed[str(docContainingAWord)]=1
    



    for docTobeAnalyzed in docsToBeAnalyzed:
      if int(docTobeAnalyzed)> data.shape[1]:
        print("HALT, exceed")

      result.append(QueryResponse(docTobeAnalyzed,scoreWrapper(int(docTobeAnalyzed),QueryVector)))

    # BENCHMARK: GET RESULT BACK

    
    return result

    # for docID in range(len(Array)):
    """
    result=[]

    for docID in range(len(Array)):
      temp=scoreWrapper(docID)

      if temp[0]== -1:
        continue

      docFate=QueryResponse(docID,temp[0],temp[1])

      result.append(copy.deepcopy(docFate))
      result.sort()

      if len(result)==6:
        del result[0]

    return result
    """

    #   if temp[0]== -1:
    #     continue

    #   docFate=QueryResponse(docID,temp[0],temp[1])

    #   result.append(copy.deepcopy(docFate))
    #   result.sort()

    #   if len(result)==6:
    #     del result[0]

    # return result

result=retrieve()
result.sort()
result.reverse()

# print(type(result))
print(len(result)," result(s) found")


# for i in range(min(20,int(len(result)/10))):
#   print(result[i].score)


temp=0

for r in result:
  docID=r.docID
  print("docID=",docID," url=",data[int(docID)]['url'],"score=",np.round(r.score,3))
  temp+=1
  if temp==5:
    break
# temp=0

#for key in postingList:
 # print(key,)
 # temp=temp+1


Queryنرخ تسعیر

98  result(s) found
docID= 8321  url= https://www.farsnews.ir/news/14001106000448/ابلاغیه-معاون-اول-رئیس‌جمهور-برای-حمایت-از-بازار-سرمایه score= 0.353
docID= 9400  url= https://www.farsnews.ir/news/14001006000468/عضو-کمیسیون-تلفیق-‌-دولت-در-بودجه-40-توجه-ویژه‌ای-به-بازار-سرمایه score= 0.228
docID= 8933  url= https://www.farsnews.ir/news/14001019000358/مخالف-کلیات-بودجه-باید-مبتنی-بر-واقعیت-جامعه-تصمیم-گیری-کنیم score= 0.218
docID= 7954  url= https://www.farsnews.ir/news/14001117000485/ارز-4200-تومانی-برای-برخی-واردکنندگان-کالاهای-اساسی-رانت-ایجاد-کرد score= 0.191
docID= 6963  url= https://www.farsnews.ir/news/14001221000788/سیاست‌گذاری‌های-ارزی-دهه-90-همواره-در-جهت-تضعیف-ارزش-پول-ملی-حرکت score= 0.179
